In [4]:
search_for = 124
start_from = 0
threads = 10
thread = 2
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004314899444580078
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 75
all cases: 621252555
type: [1, 1, 1, 1, 124] 124
cases of this type: 236421376
100000 31.90392255498262
200000 48.366794631572795
300000 46.827148990894464
400000 46.77227176941491
500000 48.814552340086756
600000 47.378877238675656
700000 45.65352981954257
800000 46.178317541030616
900000 46.196468338989604
1000000 16.01279127979878
1100000 47.535036869127126
1200000 45.69901541777486
1300000 45.36801013102943
1400000 46.292330947299625
1500000 48.453422378512265
1600000 46.33252930441601
1700000 46.4408606333335
1800000 46.808047647692774
1900000 40.165262916499145
2000000 22.80350160283037
2100000 44.206248333185634
2200000 43.636440616224526
2300000 43.464995685010386
2400000 43.74248249872604
2500000 43.65143926880612
2600000 42.732218795887114
2700000 42.7533201607053
2800000 42.66732361893886
2900000 17.785386753169046
3000000 41.77

29700000 42.93009883492048
29800000 39.74568245019553
29900000 41.16835211159515
30000000 43.11257896782589
30100000 44.20202899947509
30200000 44.03122866165511
30300000 27.16581547921548
30400000 39.02581301393216
30500000 44.41248134692899
30600000 46.086874308795885
30700000 45.98911414073496
30800000 20.011953734090643
30900000 46.467344201562696
31000000 47.094885990240854
31100000 45.125878741525234
31200000 45.24197394558457
31300000 41.355481976133596
31400000 40.67029230274753
31500000 16.987337398023644
31600000 42.12124088841517
31700000 40.18642956698735
31800000 43.37348879081544
31900000 44.44544916867726
32000000 45.550702428796974
32100000 46.19061766320866
32200000 24.23193254354722
32300000 43.10644323356469
32400000 45.31731114766087
32500000 44.95098395345052
32600000 44.67910298635013
32700000 26.330876619089953
32800000 38.86755762445689
32900000 45.6363747208196
33000000 42.14667903411722
33100000 42.48072783226896
33200000 40.13383902221229
33300000 38.61351402

59800000 37.076565341676485
59900000 36.53150169028002
60000000 36.88007462447903
60100000 14.499989688231997
60200000 37.2628285900808
60300000 36.25734706573418
60400000 36.897708958017134
60500000 37.00962219367729
60600000 19.73501475703137
60700000 38.859505510199334
60800000 40.50825591625742
60900000 40.10378462376428
61000000 43.510009485061
61100000 42.87431059348673
61200000 42.118665741574944
61300000 41.85483860317544
61400000 41.03395246126293
61500000 31.88059134770416
61600000 24.319327829924834
61700000 35.940067451762765
61800000 38.036709990104555
61900000 36.29219935834227
62000000 17.09828791436465
62100000 34.302384636722074
62200000 37.766338684651295
62300000 35.328274421171464
62400000 36.52389435732115
62500000 21.13810971286608
62600000 38.886497018992955
62700000 39.0414712394771
62800000 40.597378081367864
62900000 40.47060629002444
63000000 40.23009533847178
63100000 40.55165546515853
63200000 39.67478282506762
63300000 37.60914744430828
63400000 38.0045104

89900000 35.65282516153346
90000000 36.80868648968799
90100000 35.218792825598754
90200000 34.715946948296214
90300000 36.51827869531707
90400000 16.739623691363192
90500000 32.27683991447552
90600000 17.077570567732273
90700000 33.50850203104976
90800000 26.55265482907459
90900000 27.216394501825132
91000000 36.27077016079691
91100000 34.14123295058525
91200000 37.50987003864103
91300000 34.544847356424235
91400000 38.75476018466341
91500000 38.12012668263048
91600000 39.504898756531524
91700000 42.29067577513814
91800000 38.02711643761161
91900000 38.42948053201731
92000000 36.600921879462305
92100000 39.101372747599086
92200000 35.7183622268806
92300000 16.304424959181837
92400000 39.80716599674184
92500000 17.265139694744803
92600000 31.730265738728196
92700000 25.824092582907827
92800000 30.80476605948571
92900000 35.24250225113167
93000000 38.597474084367555
93100000 36.1237100201015
93200000 38.00585490798563
93300000 39.37982452917096
93400000 41.05506937653675
93500000 41.0166

119200000 3.735838968154629
119300000 3.749331171554416
119400000 3.7884838333962856
119500000 3.8030714454933543
119600000 3.7835384293493433
119700000 3.809343806213647
119800000 3.7679712565764487
119900000 3.7800323222054675
120000000 3.7558199461281103
120100000 3.765888283134888
120200000 3.746695827035818
120300000 3.7770974059366282
120400000 3.7038450757237875
120500000 3.728104569173088
120600000 3.7545411077377397
120700000 3.7153172880443175
120800000 3.709250183937421
120900000 3.735120367732704
121000000 3.7141841935375597
121100000 3.7733885131860307
121200000 3.7541301769214277
121300000 3.7525712012866173
121400000 3.6837930396848386
121500000 3.6856760463273526
121600000 3.7283111637581454
121700000 3.94177102772571
121800000 3.75000740778319
121900000 3.73091187357881
122000000 3.742417651337296
122100000 3.7342866229893565
122200000 3.7308322863194214
122300000 3.727882960099621
122400000 3.739606228291323
122500000 3.688719376266436
122600000 3.742779244831625
1227

...

493100000 3.8750767741959593
493200000 3.8140534860668582
493300000 4.267661929298162
493400000 1.4430013968071937
493500000 3.3754761380801
493600000 3.0425851301247375
493700000 2.802636597855068
493800000 3.376496251748353
493900000 3.725113925437046
494000000 4.358397200672444
494100000 4.033726262958438
494200000 3.4562326474241947
494300000 2.3561596644830143
494400000 2.4244200864501497
494500000 3.178345818414112
494600000 2.8623524952300854
494700000 3.67303633440274
494800000 3.6077845598746334
494900000 3.6887117254629667
495000000 3.4865537073647084
495100000 3.815838199619651
495200000 4.3149876332722075
495300000 1.2653652396318713
495400000 3.281874701110168
495500000 2.231834804162595
495600000 3.3642277232891025
495700000 3.406121720560577
495800000 3.5238740539438127
495900000 4.092966039596399
496000000 3.4021074361400045
496100000 3.7534344745120505
496200000 2.450520407673071
496300000 2.3621549680409197
496400000 2.556944231128057
496500000 3.170795626464486
49660

521100000 2.9813395373494727
521200000 2.9454428884824346
521300000 2.860542434782509
521400000 2.9733783398845297
521500000 3.0900720369482237
521600000 3.0384983080716963
521700000 2.8732793984883127
521800000 2.9639942999644147
521900000 3.088238999326905
522000000 3.319493615526279
522100000 3.09326070338176
522200000 3.106367799937788
522300000 3.1610275825335084
522400000 3.2606771062258315
522500000 3.1666647158659402
522600000 3.074949567128718
522700000 3.101238748425507
522800000 3.258700840767254
522900000 3.1476164903549058
523000000 3.2239082519420283
523100000 3.0318722822667588
523200000 3.2046835688298247
523300000 3.1100098572674555
523400000 3.09612889427898
523500000 2.8012109308575686
523600000 2.755522073520124
523700000 2.708208590713286
523800000 2.874511569607089
523900000 2.9615533127135567
524000000 3.0467400192674727
524100000 2.8827135855917336
524200000 2.973726546688428
524300000 2.9305078468106256
524400000 3.7772282751472295
524500000 3.4890192758673195


547900000 3.1202876338107455
548000000 1.7513596239341465
548100000 2.746308631947438
548200000 3.1402573220271006
548300000 1.0474092987030545
548400000 0.5767904820369284
548500000 0.5709229178808828
548600000 0.589259617199252
548700000 0.5661248728049099
548800000 0.5700261759728392
548900000 0.5680262982584338
549000000 0.5605221743701001
549100000 0.5647786044841111
549200000 0.5678854326906271
549300000 0.5598338296194474
549400000 0.5674236846093205
549500000 0.5596213336915208
549600000 0.5616112213017941
549700000 0.560143528665834
549800000 0.560240528837651
549900000 0.5565463550151387
550000000 0.5413070337300996
550100000 0.65423430856963
550200000 0.5575741856962443
550300000 0.5612240400072701
550400000 0.5655571332438223
550500000 0.5556285790139437
550600000 0.574038524111314
550700000 3.439429056911018
550800000 3.261300955290457
550900000 1.3887313428630068
551000000 3.3288022488414946
551100000 3.4008531940349536
551200000 2.73620695518953
551300000 2.3239717832119

576300000 1.2651614082235694
576400000 1.2225552440583707
576500000 1.025920701761259
576600000 0.7336612831884921
576700000 1.375920788536575
576800000 1.2876072182310918
576900000 1.025478890166511
577000000 1.1813396192167633
577100000 0.5790781307879124
577200000 1.287085110223055
577300000 1.4565663814393117
577400000 1.011822767708146
577500000 1.1289590742720366
577600000 0.6769297983772523
577700000 1.294316449069967
577800000 1.1234439739943343
577900000 1.040723351616141
578000000 1.1714038859593339
578100000 0.9003735999061964
578200000 1.3024647434234056
578300000 0.9306915108551848
578400000 1.2216937191191615
578500000 0.741864258296248
578600000 1.106419268582076
578700000 1.5304949739863078
best so far: 0
type: [1, 2, 1, 31, 2] 124
cases of this type: 953312
578800000 0.4198327155523168
578900000 0.28031553594146835
579000000 0.27912642258856696
579100000 0.2824697651003732
579200000 0.28666666026368404
579300000 0.28403969906836746
579400000 0.28303288996167647
5795000

599800000 0.4182474617122246
599900000 0.7815569738951987
600000000 0.9019920842634142
600100000 0.9029967229716116
600200000 0.5936005259735617
600300000 0.8270155322110155
600400000 0.54943263643883
600500000 0.8428388460398117
600600000 0.7528963217412431
600700000 0.6328965076642765
600800000 0.8377737657807238
600900000 0.5458057205847203
601000000 0.8239483080876139
601100000 0.6204782129680216
601200000 0.720496211162438
601300000 0.7306019182895588
601400000 0.6031690576653613
601500000 0.7918443211852212
601600000 0.5659462878009015
601700000 0.7757801474641297
601800000 0.6600774120689035
601900000 0.6475850621947911
602000000 0.7444529140020808
602100000 0.5854631593535146
602200000 0.8230201082764467
602300000 0.5959481159327328
602400000 0.7064900063079099
602500000 0.7120861375491057
602600000 0.5857031802620193
602700000 0.7637901184217035
602800000 0.6184872840508123
602900000 0.7193756945595478
603000000 0.5769842631721794
603100000 0.6576543578969803
603200000 0.69915